# 05 - Evaluation and Visualization

## Purpose
Compare model predictions against actual values and visualize performance

## Code

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

preds = model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}')

plt.figure(figsize=(10,5))
plt.plot(y_test.values, label='Actual')
plt.plot(preds, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Returns')
plt.show()